<a href="https://colab.research.google.com/github/codingercat/machine-learning-projects/blob/main/Sports_Image_Classification_using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Step 1: Read CSV File
csv_path = '/content/drive/My Drive/Sports Image Classification/sports.csv'

In [ ]:
import pandas as pd

try:
    df = pd.read_csv(csv_path)
    print("CSV file loaded successfully.")
except FileNotFoundError:
    print("CSV file not found. Please verify the file path:", csv_path)


In [ ]:
# Step 2: Data Preprocessing
train_dir = '/content/drive/My Drive/Sports Image Classification/train'
test_dir = '/content/drive/My Drive/Sports Image Classification/test'
valid_dir = '/content/drive/My Drive/Sports Image Classification/valid'

In [ ]:
# Step 3: Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Step 4: Building the Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization

In [ ]:
## CNN IMPLEMENTATION ##
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu'),  # New convolutional layer
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(256, activation='relu'),  # New dense layer
    Dense(100, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Step 5: Training the Model
batch_size = 16
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
checkpoint_path = 'model_checkpoint.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# Add the checkpoint callback to the list of callbacks in model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint]
)

# Save the model after training is complete
model.save('final_model.h5')


Epoch 1/5
844/844 [==============================] - ETA: 0s - loss: 4.4638 - accuracy: 0.0241
Epoch 1: val_loss improved from inf to 4.15887, saving model to model_checkpoint.h5
844/844 [==============================] - 3276s 4s/step - loss: 4.4638 - accuracy: 0.0241 - val_loss: 4.1589 - val_accuracy: 0.0524
Epoch 2/5
229/844 [=======>......................] - ETA: 14:19 - loss: 4.1454 - accuracy: 0.0502

In [ ]:
# Step 6: Model Evaluation
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("Test accuracy:", test_acc)